# General Model Analysis

**Purpose:** Primary workspace for ad-hoc deep dives, model introspection, and feature correlation analysis.
**Author:** Roo Code
**Date:** 2026-02-16

## Setup
This notebook uses the `ml_heating` package directly.

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

# Ensure project root is in path for src imports
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))
if project_root not in sys.path:
    sys.path.append(project_root)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta, timezone

# Standard Library Imports
from src import config
from src.analysis import DataLoader, plotting
from src.thermal_equilibrium_model import ThermalEquilibriumModel

# Configure Plotting
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Data Loading
Fetch historical data for analysis. Default is the last 7 days.

In [ ]:
# Initialize Loader
loader = DataLoader()

# Define Time Range
end_time = datetime.now(timezone.utc)
start_time = end_time - timedelta(days=7)

print(f"Fetching data from {start_time} to {end_time}...")

# Fetch Data
df = loader.fetch_training_data(
    start_time=start_time,
    end_time=end_time
)

print(f"Loaded {len(df)} rows")
df.head()

## 2. Feature Correlation Analysis
Analyze how different features (Outdoor Temp, Outlet Temp, etc.) correlate with each other.

In [ ]:
if not df.empty:
    plt.figure(figsize=(10, 8))
    sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
    plt.title("Feature Correlation Matrix")
    plt.show()
else:
    print("No data available for correlation analysis.")

## 3. Model Simulation
Run the `ThermalEquilibriumModel` on the historical data to see how it would have behaved.

In [ ]:
if not df.empty:
    # Initialize Model
    model = ThermalEquilibriumModel()
    
    # Prepare results container
    simulation_results = []
    
    for idx, row in df.iterrows():
        # Extract inputs
        outdoor_temp = row.get('outdoor_temperature')
        indoor_temp = row.get('indoor_temperature')
        
        if pd.notna(outdoor_temp) and pd.notna(indoor_temp):
            # Predict required outlet temperature
            # Note: This is a simplified simulation assuming target indoor temp is constant or derived
            target_indoor = row.get('ml_target_temperature', 21.0)
            
            # Calculate required power/outlet temp (simplified)
            # In a real scenario, we would use the full state
            
            # For now, let's just inspect the model's internal parameters
            # Note: ThermalEquilibriumModel stores parameters as attributes, not a dict
            params = {
                'thermal_time_constant': model.thermal_time_constant,
                'heat_loss_coefficient': model.heat_loss_coefficient,
                'outlet_effectiveness': model.outlet_effectiveness
            }
            
            simulation_results.append({
                'timestamp': idx,
                'outdoor_temp': outdoor_temp,
                'indoor_temp': indoor_temp,
                'target_indoor': target_indoor,
                **params
            })
            
    sim_df = pd.DataFrame(simulation_results).set_index('timestamp')
    print(f"Simulated {len(sim_df)} steps")
    sim_df.head()
else:
    print("No data for simulation.")

## 4. Visualization
Visualize the relationship between Outdoor Temperature and Actual Outlet Temperature.

In [ ]:
if not df.empty and 'outdoor_temperature' in df.columns and 'outlet_temperature' in df.columns:
    plt.figure(figsize=(12, 6))
    plt.scatter(df['outdoor_temperature'], df['outlet_temperature'], alpha=0.5, label='Actual Data')
    plt.xlabel('Outdoor Temperature (°C)')
    plt.ylabel('Outlet Temperature (°C)')
    plt.title('Heating Curve: Outdoor vs Outlet Temperature')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("Missing required columns for visualization.")

## 5. Conclusions
Document your observations here.